Steps:
1. read the chapter
2. return character name with profile
3. generate a image for each character (used for consistency across chapters)
4. break the chapter content into screenplay, a collection of scenes
5. generate video for each scene with contextual input.

In [40]:
import boto3
import json
import base64
from PIL import Image
import io
import os
from botocore.config import Config
import random
import time
import uuid
from boto3.dynamodb.conditions import Key
import subprocess
import tempfile
import shutil
import re
from typing import List
import math

### function to get characters introduced in a story chapter

In [ ]:
def get_characters_with_visual_profile(chapter_text):

    client = boto3.client("bedrock-runtime")
    system = [{ "text": "You are an expert in analyzing fictional stories" }]

    text = f"""
          Extract the characters introduced in this chapter text. Also provide a brief summary of their most likely visual looks. 
          You can add imaginative visual accentuations to their physical looks so that the readers can perceive it better. The chapter text is:
          {chapter_text}
          \n\n
          The response should only have the output as a pure json object with keys character_name, character_description. NO ADDITIONAL OUTPUT SHOULD BE PRINTED.
          """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.7}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-micro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to read chapter text

In [ ]:
def get_story_chapter_text(chapter_file_with_path):

    with open(chapter_file_with_path, 'r') as file:
        chapter_content = file.read()
        return chapter_content

### function to generate image variations using Nova

In [ ]:
def generate_image_variation_using_nova(input_image_path, prompt_text, story_so_far, scene_number, output_image_directory):

    def generate_seed():
        return random.randint(10000, 99999)

    model_id = 'amazon.nova-canvas-v1:0'
    with open(input_image_path, "rb") as image_file:
        input_image = base64.b64encode(image_file.read()).decode('utf8')

    final_prompt_text = f"""
                        Generate a photo-realistic, 8k, HDR image based on the following prompt: {prompt_text}.
                        
                        Please use the story background provided below to add more detail and context to the output image.
                        
                        Story Background:
                        {story_so_far}
                        """
    body = json.dumps({
        "taskType": "IMAGE_VARIATION",
        "imageVariationParams": {
            "text": final_prompt_text,
            "negativeText": "bad quality, low resolution, cartoon",
            "images": [input_image],
            "similarityStrength": 0.7,
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 720,
            "width": 1280,
            "cfgScale": 8.0,
            "seed": generate_seed()
        }
    })

    bedrock = boto3.client(service_name='bedrock-runtime', 
                           config=Config(read_timeout=300))

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )

    response_body = json.loads(response.get("body").read())
    base64_image = response_body.get("images")[0]
    base64_bytes = base64_image.encode('ascii')
    image_bytes = base64.b64decode(base64_bytes)

    image = Image.open(io.BytesIO(image_bytes))

    image_filename = str(scene_number) + '.jpeg'
    image_path = os.path.join(output_image_directory, image_filename)
    image.save(image_path, 'JPEG')

### function to generate image using nova

In [ ]:
def image_generate_using_nova(character_name, prompt_text, output_directory):

    def format_file_name(name):
        if ' ' not in name:
            return name.lower()
        else:
            return name.lower().replace(' ', '_')

    model_id = 'amazon.nova-canvas-v1:0'
    prompt_text = 'realistic editorial photo for the following profile. ' + prompt_text
    body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt_text
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 720,
            "width": 1280,
            "cfgScale": 8.0,
            "seed": 0
        }
     })

    bedrock = boto3.client(service_name='bedrock-runtime')

    accept = "application/json"
    content_type = "application/json"

    print('generating image for', character_name)

    response = bedrock.invoke_model(body=body, modelId=model_id, accept=accept, 
                                    contentType=content_type)

    response_body = json.loads(response.get("body").read())

    base64_image = response_body.get("images")[0]
    base64_bytes = base64_image.encode('ascii')
    image_bytes = base64.b64decode(base64_bytes)

    image = Image.open(io.BytesIO(image_bytes))

    image_filename = format_file_name(character_name) + '.jpeg'
    image_path = os.path.join(output_directory, image_filename)
    image.save(image_path, 'JPEG')

### function to get the subject person from the screenplay text

In [ ]:
def get_subject_entity(scene_text):

    retry_config = Config(
        retries = {'max_attempts': 10,
                   'mode': 'adaptive'})

    client = boto3.client("bedrock-runtime", 
                           config=Config(read_timeout=300,
                                         retries=retry_config.retries))

    system = [{"text": "You are an expert in identifying the principal entity or the subject from a given sentence."}]

    text = f"""
            You are an expert in identifying the principal entity or the subject from a given sentence. 
            Based on the sentence below, output ONLY the main subject entity, which MUST be a proper noun (name of a person). 
            Do NOT return common nouns or any other type of words.

            Example:
            "Elena Martin shakes hands with Julien Moreau." -> "Elena Martin"
            "The cat jumps over the fence." -> "None" (since there is no proper noun)
            "Julien Moreau travels to Paris." -> "Julien Moreau"
            "A sketchbook was pulled out by Elena Martin." -> "Elena Martin"

            If there are multiple proper nouns, return the first one encountered.
            If no proper noun is found, return "None".

            The output should ONLY be the main subject entity. NO ADDITIONAL DESCRIPTION or introduction is required.

            Input Text:
            {scene_text}

            Focus on precision, clarity, and visual singularity in scene extraction.
            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.5}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to generate screen play summary

In [ ]:
def get_screen_play_summary(scene_description_master):

    retry_config = Config(
        retries = {'max_attempts': 10,
                   'mode': 'adaptive'})

    client = boto3.client("bedrock-runtime", 
                          config=Config(read_timeout=300, 
                                        retries=retry_config.retries))

    system = [{"text": "You are an expert creating short but concise summaries from screen play text."}]

    text = f"""
            You are an expert creating short but concise summaries from screen play description. 
            Based on the description below create a short summary. 

            The output should ONLY be the summary text. No additional description or introduction is required.

            Input Text:
            {scene_description_master}

            Focus on precision, clarity, and visual singularity in scene extraction.
            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.5}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to generate screen play from chapter text

In [ ]:
def get_screen_play_scene_generation_text(chapter_text):

    client = boto3.client("bedrock-runtime")
    system = [{ "text": "You are an expert cinematographer who can create screen plays from a novel" }]

    text = f"""
            You are an expert visual story analyst specializing in breaking down narrative text into granular, single-action visual scenes.
            
            Critical Rule: EACH SCENE MUST REPRESENT EXACTLY ONE HUMAN ACTION
            - If a sentence contains multiple actions, SPLIT IT INTO SEPARATE SCENES
            - Each scene should capture a discrete, singular human movement or interaction
            - Avoid compound or complex actions in a single scene description
            - Ensure that each scene description includes the names of the characters involved
            - Avoid including sensory details such as sounds, smells, or tastes
            - Avoid using pronouns; use character names instead
            - Write each scene description in the passive voice
            - Avoid including dialogue
            - Ensure that the sentence construction is clear and concise
            
            Objective: Transform a narrative text into a precise sequence of individual, actionable visual moments.
            
            Scene Extraction Guidelines:
            1. Isolate and extract SINGLE, DISTINCT human actions
            2. Ensure each scene represents one clear, observable action
            3. Maintain chronological sequence of actions
            4. Use the names of the characters in each scene description
            5. Exclude sensory details such as sounds, smells, or tastes
            6. Avoid using pronouns; use character names instead
            7. Write each scene description in the passive voice
            8. Avoid including dialogue
            9. Ensure that the sentence construction is clear and concise
            
            Examples of Proper Splitting:
            - INCORRECT: "John walked into the room and sat down."
            - CORRECT: 
              1. "The room was entered by John."
              2. "The chair was sat upon by John."
            
            - INCORRECT: "Their eyes met"
            - CORRECT: "Eyes met between Julien Moreau and Elena Martin."
            
            - INCORRECT: "The door chimed again."
            - CORRECT: "The café was entered by Julien Moreau."
            
            - INCORRECT: "They spoke of art, literature, their dreams, and their aspirations."
            - CORRECT: "Art, literature, dreams, and aspirations were discussed between Elena Martin and Julien Moreau."
            
            What to Include:
            - Precise, individual human movements
            - Specific character interactions
            - Singular physical actions
            - Clear, observable changes in character position or state
            - Names of the characters involved in each action
            
            What to Exclude:
            - Compound actions
            - Internal thoughts
            - Background descriptions
            - Atmospheric elements
            - Multiple actions in one scene
            - Sensory details such as sounds, smells, or tastes
            - Pronouns; use character names instead
            - Dialogue
            
            Output Requirements:
            - Structured as a JSON array of single-action scenes
            - Each scene must represent ONE human action
            - Sequentially ordered
            - Visually descriptive
            - Include character names in each scene description
            - Exclude sensory details, pronouns, and dialogue
            - Write each scene description in the passive voice
            - Ensure that the sentence construction is clear and concise
            
            Input Text:
            {chapter_text}
            
            Output Format:
                [
                    {{"sequence": "1", "scene_description": "Specific single human action"}},
                    {{"sequence": "2", "scene_description": "Next single human action"}},
                    ...
                ]
            Focus on precision, clarity, and visual singularity in scene extraction.
            """

    messages = [
        {"role": "user", "content": [{"text": text}]},
    ]

    inf_params = {"maxTokens": 3000, "topP": 0.7, "temperature": 0.7}

    additionalModelRequestFields = {
        "inferenceConfig": {
             "topK": 100
        }
    }

    model_response = client.converse(
        modelId="us.amazon.nova-pro-v1:0", 
        messages=messages, 
        system=system, 
        inferenceConfig=inf_params,
        additionalModelRequestFields=additionalModelRequestFields
    )

    return model_response["output"]["message"]["content"][0]["text"]

### function to generate character visual profile

In [ ]:
def generate_character_visual_profile(character_list_collection):

    if character_list_collection is not None:
        image_output_dir = 'pocketfm/character-profile-image'
        character_list_collection_json = json.loads(character_list_collection)
        for character in character_list_collection_json['characters']:
            character_name = character['character_name']
            character_description = character['character_description']
            image_generate_using_nova(character_name, character_description, image_output_dir)

### utility functions

In [ ]:
def format_string(input_string):
    lower_case_string = input_string.lower()
    converted_string = lower_case_string.replace(" ", "_")
    return converted_string


def upload_image_to_s3(bucket_name, object_key, file_path):
    s3 = boto3.client('s3')
    response = s3.upload_file(file_path, bucket_name, object_key)
    return response


def update_ddb_record(scene_number, scene_description, s3_image_path, previous_scene_summary):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    segment_id = str(uuid.uuid4())

    response = table.update_item(
        Key={
            'segment_id': segment_id
        },
        UpdateExpression="set scene_description = :sd, s3_image_path = :sip, previous_scene_summary = :ssmry, sequence_number = :sqnum",
        ExpressionAttributeValues={
            ':sd': scene_description,
            ':sip': s3_image_path,
            ':ssmry': previous_scene_summary,
            ':sqnum': scene_number
        },
        ReturnValues="UPDATED_NEW"
    )

    return response

### loop through the screen play segments

In [ ]:
def get_screen_play_segment_collection(max_sequence=None):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('pocketfm_screen_play_scene_map')
    scan_kwargs = {
        'FilterExpression': Key('sequence_number').gte(1),  
        'ProjectionExpression': 'segment_id, scene_description, s3_image_path, sequence_number'
    }
    if max_sequence:
        scan_kwargs['FilterExpression'] &= Key('sequence_number').lte(max_sequence)
    response = table.scan(**scan_kwargs)
    sorted_records = sorted(
        response.get('Items', []), 
        key=lambda x: int(x['sequence_number']))

    return sorted_records

### generate video using image (nova)

In [ ]:
def generate_video_using_nova(base_image_with_path, base_prompt, context_prompt, s3_bucket, s3_video_prefix):

    def generate_seed():
        return random.randint(10000, 99999)

    retry_config = Config(retries={'max_attempts': 10, 'mode': 'adaptive'})

    bedrock_runtime = boto3.client("bedrock-runtime", 
                                   config=Config(read_timeout=300, retries=retry_config.retries))

    with open(base_image_with_path, "rb") as f:
        input_image_bytes = f.read()
        input_image_base64 = base64.b64encode(input_image_bytes).decode("utf-8")

    prompt = f"""
             Cinematic Dolly Shot of {base_prompt}
             """
    model_input = {
        "taskType": "TEXT_VIDEO",
        "textToVideoParams": {
            "text": prompt,
            "images": [
                {
                    "format": "png",
                    "source": {
                        "bytes": input_image_base64
                    }
                }
            ]},
        "videoGenerationConfig": {
            "durationSeconds": 6,
            "fps": 24,
            "dimension": "1280x720",
            "seed": generate_seed()
        },
    }
    s3_uri = f"s3://{s3_bucket}/{s3_video_prefix}"
    invocation = bedrock_runtime.start_async_invoke(modelId="amazon.nova-reel-v1:0", 
                                                    modelInput=model_input,
                                                    outputDataConfig={
                                                        "s3OutputDataConfig": {
                                                            "s3Uri": s3_uri
                                                        }
                                                    },)
    print("Response:")
    print(json.dumps(invocation, indent=2, default=str))

    invocation_arn = invocation["invocationArn"]
    while True:
        invocation = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
        output_s3_bucket_with_prefix = invocation["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
        status = invocation["status"]
        if (status == "Completed"):
            print('video is ready for review...')
            break
        elif (status == "InProgress"):
            print('video generation in progress in the s3 bucket', output_s3_bucket_with_prefix)
            time.sleep(5)
        elif (status == "Failed"):
            print('video generation failed. better luck next time..')
            break

    return output_s3_bucket_with_prefix

In [ ]:
# # test video generation
# base_image_with_path = 'pocketfm/scene-cover-image/1.jpeg'
# base_prompt = 'A quaint café was stumbled upon by Elena Martin.'
# context_prompt = ''
# s3_bucket = 'pocket-fm-poc'
# s3_video_prefix = 'video-generator/output-video/'
# generate_video_using_nova(base_image_with_path, base_prompt, context_prompt, s3_bucket, s3_video_prefix)

In [ ]:
# retry_config = Config(retries={'max_attempts': 10, 'mode': 'adaptive'})

# bedrock_runtime = boto3.client("bedrock-runtime", 
#                                config=Config(read_timeout=300, retries=retry_config.retries))
# invocation = bedrock_runtime.get_async_invoke(
#     invocationArn="arn:aws:bedrock:us-east-1:034049775477:async-invoke/jcodv7pta3hv"
# )
# print(invocation)

### generate visualization for screen play

In [ ]:
def generate_visualization_for_screen_play(scene_collection, character_profile_image_dir, s3_prefix, scene_cover_image_dir):

    scene_collection_list = json.loads(scene_collection)
    scene_description_master = ''
    for scene in scene_collection_list:
        sequence_number = int(scene['sequence'])
        scene_description = scene['scene_description']
        print(f'processing {sequence_number} and description {scene_description}')
        # get the seed image to choose
        subject_entity = get_subject_entity(scene_description)
        subject_entity_formatted = format_string(subject_entity)
        seed_image_input_with_path = character_profile_image_dir + subject_entity_formatted + '.jpeg'
        # get summary of the screenplay till previous iteration
        screen_play_summary = get_screen_play_summary(scene_description_master)
        # generate cover image for this scene

        # the following line is a hack to circumvent the guardrail in bedrock of a specific person
        scene_description = scene_description.replace("Elena Martin", " the woman ").replace("Julien Moreau", " the man ")

        generate_image_variation_using_nova(seed_image_input_with_path, 
                                            scene_description,
                                            screen_play_summary,
                                            sequence_number,
                                            scene_cover_image_dir)
        # upload the image to a s3 bucket
        s3_object_key = f"{s3_prefix}{subject_entity_formatted}_{sequence_number}.jpeg"
        generated_image_path = scene_cover_image_dir + str(sequence_number) + '.jpeg'
        upload_image_to_s3(s3_bucket, s3_object_key, generated_image_path)
        # save record to DDB
        update_ddb_record(sequence_number, scene_description, s3_object_key, screen_play_summary)
        # append to the screenplay summary
        scene_description_master = scene_description_master + scene_description
        # avoid throtlling
        time.sleep(5)

### Generate video segments from screen play segments

In [ ]:
def generate_video_segments_from_screen_play_segments(bucket_name, 
                                                      local_image_directory, 
                                                      context_prompt, 
                                                      s3_video_prefix, 
                                                      vid_duration_seconds=6):

    def download_s3_image(bucket_name, s3_image_path, local_directory=None):
        s3_client = boto3.client('s3')
        parts = s3_image_path.split('/')
        s3_key = '/'.join(parts)

        filename = parts[-1]
        local_path = os.path.join(local_directory, filename)

        s3_client.download_file(Bucket=bucket_name, Key=s3_key, Filename=local_path)

        return local_path

    def calculate_segment_times(sequence_number, segment_duration=6):
        start_time = (sequence_number - 1) * segment_duration
        end_time = start_time + segment_duration

        return start_time, end_time

    def update_screen_play_segment_video(segment_id, video_url, start_time_seconds, end_time_seconds):
        dynamodb = boto3.resource('dynamodb')
        table = dynamodb.Table('pocketfm_screen_play_scene_map')
        table.update_item(
            Key={'segment_id': segment_id},
            UpdateExpression='SET video_url = :url, start_time_seconds = :start, end_time_seconds = :end',
            ExpressionAttributeValues={':url': video_url, 
                                       ':start': start_time_seconds, 
                                       ':end': end_time_seconds}
        )

    screen_play_segments = get_screen_play_segment_collection()
    for screen_play_segment in screen_play_segments:
        segment_id = screen_play_segment['segment_id']
        scene_description = screen_play_segment['scene_description']
        s3_image_path = screen_play_segment['s3_image_path']
        sequence_number = str(int(screen_play_segment['sequence_number']))
        print('now processing sequence number', sequence_number)
        segment_start_time, segment_end_time = calculate_segment_times(int(sequence_number))
        # download scene cover image to local folder
        local_image_path = download_s3_image(bucket_name,
                                             s3_image_path,
                                             local_image_directory)
        # invoke video generation and get output video_file
        output_s3_bucket_with_prefix = generate_video_using_nova(local_image_path,
                                                                 scene_description,
                                                                 context_prompt,
                                                                 bucket_name,
                                                                 s3_video_prefix)
        # nova names the output video files as output.mp4 by default
        output_video_file_with_s3_path = output_s3_bucket_with_prefix + '/output.mp4'
        # update ddb with the video location in s3
        update_screen_play_segment_video(segment_id, 
                                         output_video_file_with_s3_path, 
                                         segment_start_time, 
                                         segment_end_time)

### Stitch video segments

In [9]:
def stitch_video_segments(output_path='pocketfm/video-out/consolidated_video.mp4'):
    dynamodb = boto3.resource('dynamodb')
    s3_client = boto3.client('s3')
    temp_dir = tempfile.mkdtemp()
    
    try:
        table = dynamodb.Table('pocketfm_screen_play_scene_map')
        response = table.scan()
        
        sorted_records = sorted(response['Items'], 
                                key=lambda x: int(x['sequence_number']))

        video_segments = []
        for record in sorted_records:
            s3_url = record['video_url']
            bucket, key = s3_url.replace('s3://', '').split('/', 1)
            local_filename = os.path.join(temp_dir,
                                          f"segment_{record['sequence_number']}.mp4")
            s3_client.download_file(bucket, key, local_filename)
            video_segments.append(local_filename)
        
        input_file_path = os.path.join(temp_dir, 'input_list.txt')
        with open(input_file_path, 'w') as f:
            for video in video_segments:
                f.write(f"file '{video}'\n")
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        ffmpeg_command = [
            'ffmpeg', 
            '-y',  
            '-loglevel', 'error', 
            '-f', 'concat', 
            '-safe', '0', 
            '-i', input_file_path, 
            '-c', 'copy', 
            output_path
        ]
        subprocess.run(ffmpeg_command, check=True)
        
        print(f"Consolidated video created: {output_path}")
        
        return output_path
    
    finally:
        # Always remove temporary directory
        shutil.rmtree(temp_dir, ignore_errors=True)

## DRIVER BLOCK

In [10]:
character_profile_image_dir = 'pocketfm/character-profile-image/'
scene_cover_image_dir = 'pocketfm/scene-cover-image/'
s3_bucket = 'pocket-fm-poc'
s3_prefix = 'video-generator/key-frame/'
local_image_directory = 'pocketfm/downloaded-scene-cover-image'
context_prompt = ''
s3_video_prefix = 'video-generator/output-video/'

# # step 1
# chapter_content = get_story_chapter_text('pocketfm/story/whispers-of-eternity/chapter-1.txt')

# # step # 2 - get character name and profile
# character_with_profile_collection = get_characters_with_visual_profile(chapter_content)
# generate_character_visual_profile(character_with_profile_collection)

# # step # 3
# scene_collection = get_screen_play_scene_generation_text(chapter_content)
# generate_visualization_for_screen_play(scene_collection, 
#                                        character_profile_image_dir, 
#                                        s3_prefix, 
#                                        scene_cover_image_dir)

# # step # 4
# generate_video_segments_from_screen_play_segments(s3_bucket, 
#                                                   local_image_directory,
#                                                   context_prompt,
#                                                   s3_video_prefix)

# step # 5
stitch_video_segments()

Consolidated video created: pocketfm/video-out/consolidated_video.mp4


'pocketfm/video-out/consolidated_video.mp4'

### Generate SRT file and save

In [38]:
def generate_video_sub_title_from_chapter_text(video_path, chapter_file_path):

    def get_chapter_text(chapter_file_path):
        with open(chapter_file_path, 'r', encoding='utf-8') as file:
            chapter_content = file.read()
            parts = chapter_content.split('Text:', 1)

            return parts[1].strip().replace('\n', ' ') if len(parts) > 1 else ""

    # get the video duration
    duration_cmd = ['ffprobe', 
                    '-v',
                    'error', 
                    '-show_entries', 
                    'format=duration', 
                    '-of', 'default=noprint_wrappers=1:nokey=1',
                    video_path]
    duration_output = subprocess.check_output(duration_cmd).decode().strip()
    total_duration = float(duration_output)
    print('total video duration=', total_duration)

    text = get_chapter_text(chapter_file_path)
    print(text)
    # split by sentences
    sentence_pattern = r'(?<=[.!?])\s+'
    sentences = re.split(sentence_pattern, text)
    # calculate total words and speaking time
    total_words = sum(len(sentence.split()) for sentence in sentences)
    # assumed average speaking rate
    avg_speaking_rate = 150 
    total_speaking_time = min((total_words / avg_speaking_rate) * 60, total_duration)
    print(total_words, total_speaking_time, (total_words / avg_speaking_rate) * 60)

In [39]:
generate_video_sub_title_from_chapter_text('pocketfm/video-out/consolidated_video.mp4', 'pocketfm/story/whispers-of-eternity/chapter-1.txt')

total video duration= 102.714
In the bustling heart of Paris, where the cobblestones seemed to whisper tales of lovers from years gone by, Elena Martin wandered the vibrant streets. A talented artist, Elena had come to Paris seeking inspiration for her next collection, unaware that she was about to find something even more profound than her art.   It was a crisp autumn morning, and the air was alive with the scent of freshly fallen leaves and the promise of rain. Elena’s breath misted in the cool air as she strolled through Montmartre, her eyes drinking in the beauty of the eclectic colors and the life of the city around her. As she turned a corner, she stumbled upon a quaint café, its warm glow spilling onto the cobbled street.  Drawn by the aroma of freshly baked croissants and rich coffee, she pushed open the door. A gentle chime rang out, welcoming her into the cozy embrace of the café. Elena chose a small table by the window, where the view outside offered a perfect glimpse of Par

In [14]:
def generate_subtitles_from_chapter(video_path: str, chapter_content: str) -> List[dict]:
    """
    Generate subtitles by splitting chapter content and allocating time
    
    :param video_path: Path to input video
    :param chapter_content: Full chapter text
    :return: List of subtitle dictionaries
    """
    # Get video duration
    duration_cmd = [
        'ffprobe', 
        '-v', 'error', 
        '-show_entries', 'format=duration', 
        '-of', 'default=noprint_wrappers=1:nokey=1', 
        video_path
    ]
    
    # Execute duration command
    duration_output = subprocess.check_output(duration_cmd).decode().strip()
    total_duration = float(duration_output)
    
    # More robust sentence splitting for this specific text
    def split_sentences(text):
        # Remove paragraph breaks
        text = text.replace('\n', ' ')
        
        # Split on punctuation that ends sentences
        sentence_pattern = r'(?<=[.!?])\s+'
        sentences = re.split(sentence_pattern, text)
        
        # Clean and filter sentences
        sentences = [
            s.strip() for s in sentences 
            if s.strip() and len(s.strip().split()) > 3  # Ignore very short fragments
        ]
        
        return sentences

    # Split chapter into sentences
    sentences = split_sentences(chapter_content)
    
    # Calculate total words and speaking time
    total_words = sum(len(sentence.split()) for sentence in sentences)
    avg_speaking_rate = 150  # words per minute
    total_speaking_time = min((total_words / avg_speaking_rate) * 60, total_duration)
    
    # Generate subtitles
    subtitles = []
    current_time = 0
    
    for i, sentence in enumerate(sentences):
        # Calculate time for this sentence
        words_in_sentence = len(sentence.split())
        sentence_duration = (words_in_sentence / avg_speaking_rate) * 60
        
        # Adjust duration if it would exceed video length
        if current_time + sentence_duration > total_duration:
            break
        
        # Create subtitle entry
        subtitle = {
            'index': i + 1,
            'start': _format_time(current_time),
            'end': _format_time(current_time + sentence_duration),
            'text': sentence
        }
        
        subtitles.append(subtitle)
        
        # Update current time
        current_time += sentence_duration
    
    return subtitles

def _format_time(seconds: float) -> str:
    """
    Convert seconds to SRT time format
    
    :param seconds: Time in seconds
    :return: Formatted time string
    """
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    
    return f"{hours:02d}:{minutes:02d}:{secs:06.3f}".replace('.', ',')

def write_srt_file(subtitles: List[dict], output_path: str):
    """
    Write subtitles to SRT file
    
    :param subtitles: List of subtitle dictionaries
    :param output_path: Path to output SRT file
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for subtitle in subtitles:
            f.write(f"{subtitle['index']}\n")
            f.write(f"{subtitle['start']} --> {subtitle['end']}\n")
            f.write(f"{subtitle['text']}\n\n")

# Debugging function to print out sentences
def print_sentences(chapter_content):
    def split_sentences(text):
        # Remove paragraph breaks
        text = text.replace('\n', ' ')
        
        # Split on punctuation that ends sentences
        sentence_pattern = r'(?<=[.!?])\s+'
        sentences = re.split(sentence_pattern, text)
        
        # Clean and filter sentences
        sentences = [
            s.strip() for s in sentences 
            if s.strip() and len(s.strip().split()) > 3  # Ignore very short fragments
        ]
        
        return sentences

    sentences = split_sentences(chapter_content)
    print("Total sentences found:", len(sentences))
    for i, sentence in enumerate(sentences, 1):
        print(f"{i}. {sentence}")

# Example usage
def main():
    video_path = 'input_video.mp4'
    
    # Debugging: print out all sentences
    print_sentences(chapter_content)
    
    # Generate subtitles
    subtitles = generate_subtitles_from_chapter(video_path, chapter_content)
    
    # Write to SRT file
    write_srt_file(subtitles, 'output.srt')
    
    # Print out the number of sentences processed
    print(f"Processed {len(subtitles)} sentences")

In [15]:
chapter_content = """In the bustling heart of Paris, where the cobblestones seemed to whisper tales of lovers from years gone by, Elena Martin wandered the vibrant streets. A talented artist, Elena had come to Paris seeking inspiration for her next collection, unaware that she was about to find something even more profound than her art.

It was a crisp autumn morning, and the air was alive with the scent of freshly fallen leaves and the promise of rain. Elena's breath misted in the cool air as she strolled through Montmartre, her eyes drinking in the beauty of the eclectic colors and the life of the city around her. As she turned a corner, she stumbled upon a quaint café, its warm glow spilling onto the cobbled street.

Drawn by the aroma of freshly baked croissants and rich coffee, she pushed open the door. A gentle chime rang out, welcoming her into the cozy embrace of the café. Elena chose a small table by the window, where the view outside offered a perfect glimpse of Parisian life. Pulling out her sketchbook, her fingers moved effortlessly, already eager to capture the scene before her.

As she lost herself in the strokes of her pencil, the door chimed again. This time, it announced the arrival of Julien Moreau, a writer struggling with a case of writer's block. Julien had always found solace in the warm atmosphere of the café, hoping that its calm would help spark the inspiration that had eluded him. 

Julien, with his tousled dark hair and intense eyes, carried an air of quiet contemplation. His gaze was naturally drawn to Elena as he passed by her table, their eyes locking for a brief moment that sent a spark of recognition through them both—almost as if they had known each other before. 

Ordering his usual black coffee, Julien found a seat across the room, but his focus wasn't on his work. His attention kept drifting back to Elena, her serene focus and flowing auburn hair captivating him. Something about her presence drew him in, and he couldn't shake the feeling that their paths had crossed for a reason.

Elena, too, felt the pull of the stranger's gaze. Curious and intrigued, she found herself glancing up to meet his eyes again. This time, she held his gaze a little longer, her heart quickening in her chest. There was something about him that felt familiar, a connection she couldn't quite explain.

Summoning his courage, Julien stood and approached her table. "Excuse me," he began, his voice calm yet warm, "I couldn't help but notice your drawings. They're beautiful."

Elena looked up, a soft smile forming on her lips. "Thank you," she said, her voice gentle but confident. "I'm Elena."
"Julien," he introduced himself, extending his hand. As their fingers touched, a jolt of energy passed between them, a shared recognition that something extraordinary had just begun.

For the next few hours, the café became their sanctuary. They spoke of art, literature, their dreams, and their aspirations. Elena shared her sketches, each one a window into her soul, while Julien recited some of his poetry, his words painting vivid pictures of emotion and beauty. Their connection was undeniable, a deep understanding that went beyond words.

As the day drew to a close and the sun began its descent, casting a soft, golden light over the city, they realized that something special had begun between them. A bond that felt fragile, yet strong, had formed—a love story that was only just beginning. 

When they parted ways that evening, their hearts were full of hope and anticipation, knowing that this was just the first chapter in their journey together."""


generate_subtitles_from_chapter('pocketfm/video-out/consolidated_video.mp4', chapter_content)

[{'index': 1,
  'start': '00:00:00,000',
  'end': '00:00:10,000',
  'text': 'In the bustling heart of Paris, where the cobblestones seemed to whisper tales of lovers from years gone by, Elena Martin wandered the vibrant streets.'},
 {'index': 2,
  'start': '00:00:10,000',
  'end': '00:00:21,200',
  'text': 'A talented artist, Elena had come to Paris seeking inspiration for her next collection, unaware that she was about to find something even more profound than her art.'},
 {'index': 3,
  'start': '00:00:21,200',
  'end': '00:00:30,400',
  'text': 'It was a crisp autumn morning, and the air was alive with the scent of freshly fallen leaves and the promise of rain.'},
 {'index': 4,
  'start': '00:00:30,400',
  'end': '00:00:42,400',
  'text': "Elena's breath misted in the cool air as she strolled through Montmartre, her eyes drinking in the beauty of the eclectic colors and the life of the city around her."},
 {'index': 5,
  'start': '00:00:42,400',
  'end': '00:00:50,000',
  'text': 'A